In [ ]:
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.

In [ ]:
#
#    Podaacpy integration example
#    This example integrates Podaacpy within Apache Open Climate Workbench (OCW) to download 
#    a PODACC dataset, execute some climate model evaluation and then plot a contour map.
#    
#    In this example:
#    1. Download a remote PO.DAAC (https://podaac.jpl.nasa.gov/) dataset
#       and read it into an OCW dataset object. In this case we demonstrate the source dataset
#       'Cross-Calibrated Multi-Platform Ocean Surface Wind Vector L3.0 First-Look Analyses'
#       which can be found at 
#       https://podaac.jpl.nasa.gov/dataset/CCMP_MEASURES_ATLAS_L4_OW_L3_0_WIND_VECTORS_FLK
#    2. Create a temporal Standard Deviation metric using one of the OCW standard metrics.
#    3. Evaluate the dataset against the metric and plot a contour map.
#    OCW modules demonstrated:
#    1. datasource/podaac_datasource
#    2. metrics
#    3. evaluation
#    4. plotter

In [ ]:
# imports 
# N.B. you NEED to have Apache OCW installed locally before attemtping the following imports,
# you can do this as follows 

# $  conda install -c conda-forge ocw 

from __future__ import print_function

import ocw.data_source.podaac_datasource as podaac
import ocw.evaluation as evaluation
import ocw.metrics as metrics
import ocw.plotter as plotter

In [ ]:
# Define some constants
datasetId = 'PODAAC-CCF30-01XXX'
variable = 'uwnd'
name = 'Cross-Calibrated Multi-Platform Ocean Surface Wind Vector L3.0 First-Look Analyses'
output_plot = "ccmp_temporal_standard_deviation"
download_path = "/tmp"

In [ ]:
# Step 1: Download remote PO.DAAC Dataset and read it into an OCW Dataset Object.
print("Available Level4 PO.DAAC Granules: %s" % (podaac.list_available_extract_granule_dataset_ids()))
print("Attempting to download Level4 Granule from Collection '%s' - '%s'." % (datasetId, name))
ccmp_dataset = podaac.extract_l4_granule(variable=variable, dataset_id=datasetId, name=name, path=download_path)
print("Granule details: '%s'" % (ccmp_dataset))
#print("CCMP_Dataset.values shape: (times, lats, lons) - %s \n" % (ccmp_dataset.values.shape))

# Acessing latittudes and longitudes of netCDF file
lats = ccmp_dataset.lats
lons = ccmp_dataset.lons

In [ ]:
# Step 2:  Build a Metric to use for Evaluation - Temporal Standard Deviation for this example.
# You can build your own metrics, but OCW also ships with some common metrics
print("Setting up a Temporal standard deviation metric to use for evaluation...")
std = metrics.TemporalStdDev()

In [ ]:
# Step 3: Create an Evaluation Object using Datasets and our Metric.
# The Evaluation Class Signature is:
# Evaluation(reference, targets, metrics, subregions=None)
# Evaluation can take in multiple targets and metrics, so we need to convert
# our examples into Python lists.  Evaluation will iterate over the lists
print("Making the Evaluation definition...")
# Temporal STD Metric gets one target dataset then reference dataset
# should be None
std_evaluation = evaluation.Evaluation(None, [ccmp_dataset], [std])
print("Executing the Evaluation using the object's run() method...")
std_evaluation.run()
print("Completed model evaluation.")

In [ ]:
# Step 4: Make a Plot from the Evaluation.results.
# The Evaluation.results are a set of nested lists to support many different
# possible Evaluation scenarios.
#
# The Evaluation results docs say:
# The shape of results is (num_metrics, num_target_datasets) if no subregion
# Accessing the actual results when we have used 1 metric and 1 dataset is
# done this way:
print("Accessing the Results of the Evaluation run")
results = std_evaluation.unary_results[0][0]
print("The results are of type: %s" % type(results))
print("Generating a contour map using ocw.plotter.draw_contour_map()")

fname = output_plot
gridshape = (4, 5)
plot_title = name + " Temporal Standard Deviation"
sub_titles = range(2002, 2010, 1)

plotter.draw_contour_map(results, lats, lons, fname,
                         gridshape=gridshape, ptitle=plot_title,
subtitles=sub_titles)

In [ ]:
# Finally, view the image
from IPython.display import Image
Image(filename=output_plot + '.png') 